# 🧭 Script 06 — BM25 filtré + Query Understanding (Notebook de présentation)

## 🧩 Pourquoi cette progression ?
Dans les scripts précédents, nous avons établi une **baseline BM25** puis testé deux leviers classiques d’amélioration :

- **Filtrage métier** (réduire le bruit documentaire en limitant le périmètre au *Code du travail*).
- **Query Understanding** (tenter d’**enrichir** la requête utilisateur avec des termes “métier” issus d’un dictionnaire).

L’idée de ce script est de **combiner** ces deux leviers :
1) on restreint le corpus à un périmètre juridique cohérent (*Code du travail*),  
2) puis on applique l’enrichissement de requête **avant** le retrieval BM25,  
3) et on ré-évalue **sur le même benchmark** avec **les mêmes métriques** (Recall@10, MRR, nDCG@10).

✅ Si les métriques bougent, c’est attribuable à cette combinaison (filtrage + enrichissement), à périmètre et protocole constants.


## ⚙️ Pré-requis (Notebook)
Ce notebook ajoute uniquement des cellules “setup” (installation + import du projet) pour éviter les erreurs dans Jupyter.

- Les modules `corpus_loader`, `benchmark_queries`, `metrics`, `query_understanding` sont **locaux au projet** : il faut que la racine de notre projet soit dans `sys.path`.
- Dépendances externes attendues : `rank-bm25` (et souvent `pyyaml` si le dictionnaire est en YAML).


In [1]:
# ✅ Installation des dépendances externes (à exécuter une seule fois par environnement)
# Note: dans Jupyter, %pip installe dans le kernel courant.
%pip install -q rank-bm25 pyyaml ipywidgets tqdm


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ✅ Rendre les modules locaux importables (ex: corpus_loader.py, metrics.py, etc.)
# Si notre notebook est dans un sous-dossier (ex: notebooks/), remonter d'un ou plusieurs niveaux.
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()
# Exemple si le notebook est dans un dossier notebooks/ :
# PROJECT_ROOT = Path.cwd().parent

sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: d:\-- Projet RAG Avocats --\codes_python\notebooks


## 🧾 Code du script (inchangé)

> Le code ci-dessous est repris tel quel du fichier `.py` (hors cellules setup notebook).

In [3]:
# -*- coding: utf-8 -*-

"""
STAGE 8 – BM25 + FILTRAGE MÉTIER + QUERY UNDERSTANDING
"""

from rank_bm25 import BM25Okapi
from corpus_loader import documents
from query_understanding import process_user_query, load_juridical_dictionary
from benchmark_queries import benchmark_queries
from metrics import recall_at_k, reciprocal_rank, ndcg_at_k
import re


# =========================================================
# 0. CHARGEMENT DU DICO DE juridical_dictionary.yml
# =========================================================
dictionary = load_juridical_dictionary("juridical_dictionary.yml")


# =========================================================
# 1. FILTRAGE MÉTIER (CODE DU TRAVAIL)
# =========================================================
def tokenize(text):
    return re.findall(r"\b\w+\b", text.lower())


filtered_documents = [
    doc for doc in documents
    if "code du travail" in doc["text"].lower()
]

tokenized_docs = [tokenize(doc["text"]) for doc in filtered_documents]
bm25 = BM25Okapi(tokenized_docs)


def bm25_search(query, k=10):
    scores = bm25.get_scores(tokenize(query))
    ranked = sorted(
        zip(filtered_documents, scores),
        key=lambda x: x[1],
        reverse=True
    )
    return ranked[:k]


# =========================================================
# 2. BENCHMARK
# =========================================================

def evaluate():
    recall_scores, mrr_scores, ndcg_scores = [], [], []

    for q in benchmark_queries:
        enriched = process_user_query(q["question"], dictionary)
        query = enriched["enriched_query"]

        results = bm25_search(query, k=10)

        recall_scores.append(recall_at_k(results, q["relevant_keywords"], 10))
        mrr_scores.append(reciprocal_rank(results, q["relevant_keywords"]))
        ndcg_scores.append(ndcg_at_k(results, q["relevant_keywords"], 10))

    return {
        "Recall@10": sum(recall_scores) / len(recall_scores),
        "MRR": sum(mrr_scores) / len(mrr_scores),
        "nDCG@10": sum(ndcg_scores) / len(ndcg_scores),
    }


if __name__ == "__main__":
    print("\n=== BM25 filtré + requête enrichie ===")
    print(evaluate())


Corpus brut chargé : 4422 documents
Corpus filtré 'Code du travail' : 882 documents

=== BM25 filtré + requête enrichie ===
{'Recall@10': 0.3333333333333333, 'MRR': 0.3333333333333333, 'nDCG@10': 0.3333333333333333}


## 📊 Analyse finale (comparative)

Résultat observé lors du run :

- Corpus brut chargé : **4422** documents  
- Corpus filtré **« Code du travail »** : **882** documents  
- **BM25 filtré + requête enrichie** :  
  - Recall@10 = **0.333**
  - MRR = **0.333**
  - nDCG@10 = **0.333**

### 🔍 Lecture
- Le score est **identique** à la baseline observée dans les stages BM25 précédents.
- Dans notre cas, cela s’explique très souvent par un point déjà identifié :  
  si `process_user_query()` ne détecte **pas d’intention métier** (`intent_detected=None`), alors `enriched_query == query` → **aucune variation possible** côté BM25.

### ✅ Conclusion
Ce stage est surtout utile pour **valider l’architecture expérimentale** :  
filtrage + enrichissement sont “branchés”, et on peut désormais mesurer un gain **dès que** :
- la détection d’intention se déclenche (dictionnaire plus robuste / matching moins strict),
- ou que l’on passe à une approche hybride (BM25 + dense) / re-ranking.

> À ce stade, l’absence de gain ne signifie pas que l’idée est mauvaise : elle indique que l’enrichissement *n’a pas eu d’effet* sur le benchmark actuel (souvent parce qu’il ne s’active pas).


In [4]:
# 🔁 Optionnel : relancer l'évaluation ici pour vérifier les scores dans notre environnement notebook
# (Si tout est bien installé/importé, cela doit reproduire les résultats.)
try:
    print("\n=== BM25 filtré + requête enrichie (re-run notebook) ===")
    print(evaluate())
except Exception as e:
    print("Erreur lors du re-run notebook:", repr(e))



=== BM25 filtré + requête enrichie (re-run notebook) ===
{'Recall@10': 0.3333333333333333, 'MRR': 0.3333333333333333, 'nDCG@10': 0.3333333333333333}
